In [7]:
from tg_parser import TelegramParser

async def example():
    # Использование с контекстным менеджером
    async with TelegramParser() as parser:
        # Получение сообщений из канала
        url = 'https://t.me/s/inparty_msk'
        messages = await parser.get_channel_messages(url, limit=30, reverse=True)
        return messages


# Запуск примера
ss = TelegramParser.run_async(example())

In [6]:
ss[3].message

'Taty77\n\nЕсть ли желающие поехать на горнолыжку в Кировск, по датам 6-11 декабря?\n\nЕду кататься на сноуборде в Кировск(там уже сезон открыт), поездка 6-11 декабря\n\nГород: #Москва\nКатегория: Сноуборд, Лыжи, Путешествия\nКогда: 06 декабря\n\n✉️ Написать в InParty'

In [8]:
import os
import dotenv
import json
from openai import OpenAI
from event_miner_agent import EventMinerAgent

# Загружаем переменные окружения
dotenv.load_dotenv()

# Инициализация LLM
llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Создание агента для извлечения событий
agent = EventMinerAgent(llm)

# Получаем сообщения из канала (используем ss из предыдущей ячейки)
try:
    # Обрабатываем сообщения по 3 за раз
    events = []
    batch_size = 3
    for i in range(0, len(ss), batch_size):
        batch = ss[i:i + batch_size]
        batch_events = agent.process_messages(batch)
        events.extend(batch_events)
    
    # Преобразуем события в формат, пригодный для сериализации в JSON
    events_json = []
    for event in events:
        event_dict = {
            "title": getattr(event, "title", None),
            "event_type": getattr(event, "event_type", None),
            "is_online": getattr(event, "is_online", None),
            "location": getattr(event, "location", None),
            "date": getattr(event, "date", None),
            "time": getattr(event, "time", None),
            "source_message_id": getattr(event, "source_message_id", None),
            "original_text": getattr(event, "original_text", None),
        }
        events_json.append(event_dict)
    
    # Сохраняем результат как JSON
    output_path = "test_personal_events.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(events_json, f, ensure_ascii=False, indent=2)
    
    print(f"Найдено событий: {len(events)}. Сохранено в {output_path}")
except NameError:
    print("Сначала выполните ячейку с получением сообщений (ss)")
except Exception as e:
    print(f"Ошибка при обработке: {e}")


Найдено событий: 27. Сохранено в test_personal_events.json


In [ ]:
# Пример работы с сообщениями из избранного
async def extract_events_from_saved():
    async with TelegramParser() as parser:
        # Получаем сообщения из избранного
        saved_messages = await parser.get_saved_messages(limit=20, text_only=True, reverse=True)
        
        if saved_messages:
            # Обрабатываем сообщения типа Message из telethon
            events = agent.process_messages(saved_messages)
            
            print(f"Найдено событий в избранном: {len(events)}\n")
            for i, event in enumerate(events, 1):
                print(f"Событие {i}:")
                print(f"  Название: {event.title}")
                print(f"  Тип: {event.event_type}")
                print(f"  Онлайн: {event.is_online}")
                print(f"  Место: {event.location}")
                print(f"  Дата: {event.date}")
                print(f"  Время: {event.time}")
                if event.original_text:
                    print(f"  Оригинал: {event.original_text[:100]}...")
                print()
            
            return events
        else:
            print("Нет сообщений в избранном")
            return []

# Запуск
events_from_saved = TelegramParser.run_async(extract_events_from_saved())


In [ ]:
# Пример работы с батчами (для большого количества сообщений)
async def extract_events_batch():
    async with TelegramParser() as parser:
        # Получаем много сообщений
        saved_messages = await parser.get_saved_messages(limit=100, text_only=True, reverse=True)
        
        if saved_messages:
            # Обрабатываем батчами по 10 сообщений
            events = agent.process_messages_batch(saved_messages, batch_size=10)
            
            print(f"Всего обработано сообщений: {len(saved_messages)}")
            print(f"Найдено событий: {len(events)}\n")
            
            # Группируем по типу события
            events_by_type = {}
            for event in events:
                event_type = event.event_type or "неизвестно"
                if event_type not in events_by_type:
                    events_by_type[event_type] = []
                events_by_type[event_type].append(event)
            
            print("События по типам:")
            for event_type, type_events in events_by_type.items():
                print(f"  {event_type}: {len(type_events)}")
            
            return events
        else:
            print("Нет сообщений")
            return []

# Запуск (раскомментируйте для выполнения)
# events_batch = TelegramParser.run_async(extract_events_batch())


In [2]:
saved_messages

NameError: name 'saved_messages' is not defined